In [1]:
import sys

sys.path.append("../../")

from src.ingestion.database.reader import *

reader = IngestionReader(db_host="127.0.0.1", 
                         db_user="daviswen", 
                         db_password="davis",
                         jdbc_driver_path="../../third_party/postgresql-42.5.0.jar")

22/10/25 15:58:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
users = ReadUsers(reader)
contents = ReadContents(reader)
ratings = ReadRatingFeedbacks(reader)

all_users = users.select(["id"])
all_contents = contents.select(["id"])
active_users = ratings.select(["user_id"]).distinct()
rated_contents = ratings.select(["content_id"]).distinct()

total_user_count = all_users.count()
active_user_count = active_users.count()
total_content_count = all_contents.count()
rated_content_count = rated_contents.count()
rating_count = ratings.count()

print("Total number of users: active_user={0} total={1}".format(active_user_count, total_user_count))
print("Total number of contents: rated_content={0} total={1}".format(rated_content_count, total_content_count))
print("Total number of ratings: ", rating_count)

Total number of users: active_user=283228 total=283228
Total number of contents: rated_content=53889 total=58098
Total number of ratings:  27753444


In [5]:
# Potential embeddings dimension based on the empirical ruleo of 4th root.
user_embeddings_dimension = round(pow(active_user_count, 1.0/4.0))
movie_embeddings_dimension = round(pow(rated_content_count, 1.0/4.0))

print("Potential user embeddings dimension:  ", user_embeddings_dimension)
print("Potential movie embeddings dimension: ", movie_embeddings_dimension)

Potential user embeddings dimension:   23
Potential movie embeddings dimension:  15


In [6]:
# Assuming that movie features have to come entirely from a model, 
# this gives the total number of unknown variables in the system.
system_unknowns = active_user_count*user_embeddings_dimension + rated_content_count*movie_embeddings_dimension

# Assuming we use 80% of the data for training, the number of equations we will have is:
system_equations = round(0.8*rating_count)

print("Potential number of unknown embeddings variables: ", system_unknowns)
print("Total number of equations in the system:          ", system_equations)
print("The ratio of equations to unknowns:               ", system_equations/system_unknowns)

Potential number of unknown embeddings variables:  7322579
Total number of equations in the system:           22202755
The ratio of equations to unknowns:                3.0320949763737612
